<a href="https://colab.research.google.com/github/Eldorado7621/hardware-acceleration-of-arrhytmia-detection/blob/main/cnn_cwt_an.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install tsaug

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import numpy as np

from keras import optimizers, losses, activations, models
from keras.callbacks import ModelCheckpoint, EarlyStopping, LearningRateScheduler, ReduceLROnPlateau
from keras.layers import Dense, Input, Dropout, Conv1D, Conv2D, MaxPool1D, GlobalMaxPool1D, MaxPool2D, GlobalMaxPool2D, Flatten, concatenate, Add, ReLU, Activation, BatchNormalization
from sklearn.metrics import f1_score, accuracy_score
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import tensorflow as tf

from scipy import signal

from tsaug import TimeWarp, Crop, Quantize, Drift, Reverse

import pickle
import pywt
import matplotlib.pyplot as plt

import math

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:

with open('drive/MyDrive/inputs_4class/inp_sig.pkl','rb') as f:
    X_all = pickle.load(f)
with open('drive/MyDrive/inputs_4class/labels.pkl','rb') as f:
    Y_all = pickle.load(f)

In [ ]:
Y_all = np.reshape(Y_all, ((Y_all.shape[0],1)))

all_data = np.concatenate((X_all,Y_all),axis=1)
np.random.shuffle(all_data)

indx_label4 = np.where(all_data[:,600] == 4)[0]
all_data = np.delete(all_data, indx_label4, 0)

###### 80% for train, 20% for test #########
sz = all_data.shape[0]

sz_08 = int(sz * 0.8)

In [ ]:
X_train_init = all_data[:sz_08,:600]
Y_train_init = all_data[:sz_08,600]

X_test_init = all_data[sz_08:,:600]
Y_test_init = all_data[sz_08:,600]

X_train_resamp = signal.resample(X_train_init, 256, axis=1)
X_test_resamp  = signal.resample(X_test_init, 256, axis=1)

del X_test_init
del X_train_init

CWT FUNCTION

In [ ]:

def cwt(data, wavelet_name, sampling_frequency=1.):
    
    # Currently only supported for Morlet wavelets
    if wavelet_name == 'morl':
        data -= np.mean(data)
        n_orig = data.size
        nv = 10
        ds = 1 / nv
        fs = sampling_frequency
        dt = 1 / fs

        # Pad data symmetrically
        padvalue = n_orig // 2
        x = np.concatenate((np.flipud(data[0:padvalue]), data, np.flipud(data[-padvalue:])))
        n = x.size

        # Define scales
        _, _, wavscales = getDefaultScales(wavelet_name, n_orig, ds)
        num_scales = wavscales.size

        # Frequency vector sampling the Fourier transform of the wavelet
        omega = np.arange(1, math.floor(n / 2) + 1, dtype=np.float64)
        omega *= (2 * np.pi) / n
        omega = np.concatenate((np.array([0]), omega, -omega[np.arange(math.floor((n - 1) / 2), 0, -1, dtype=int) - 1]))

        # Compute FFT of the (padded) time series
        f = np.fft.fft(x)

        # Loop through all the scales and compute wavelet Fourier transform
        psift, freq = waveft(wavelet_name, omega, wavscales)

        # Inverse transform to obtain the wavelet coefficients.
        cwtcfs = np.fft.ifft(np.kron(np.ones([num_scales, 1]), f) * psift)
        cfs = cwtcfs[:, padvalue:padvalue + n_orig]
        freq = freq * fs

        return cfs, freq
    else:
        raise Exception


def getDefaultScales(wavelet, n, ds):
    
    wname = wavelet
    nv = 1 / ds

    if wname == 'morl':

        # Smallest useful scale (default 2 for Morlet)
        s0 = 2

        # Determine longest useful scale for wavelet
        max_scale = n // (np.sqrt(2) * s0)
        if max_scale <= 1:
            max_scale = n // 2
        max_scale = np.floor(nv * np.log2(max_scale))
        a0 = 2 ** ds
        scales = s0 * a0 ** np.arange(0, 31 + 1)
    else:
        raise Exception

    return s0, ds, scales


def waveft(wavelet, omega, scales):
    
    wname = wavelet
    num_freq = omega.size
    num_scales = scales.size
    wft = np.zeros([num_scales, num_freq])

    if wname == 'morl':
        gC = 6
        mul = 2
        for jj, scale in enumerate(scales):
            expnt = -(scale * omega - gC) ** 2 / 2 * (omega > 0)
            wft[jj, ] = mul * np.exp(expnt) * (omega > 0)

        fourier_factor = gC / (2 * np.pi)
        frequencies = fourier_factor / scales

    else:
        raise Exception

    return wft, frequencies

#fs = 1000
#t = np.linspace(0, 1, fs+1)
#x = np.cos(2*np.pi*32*t) * np.logical_and(t >= 0.1, t < 0.3) + np.sin(2*np.pi*64*t) * (t > 0.7)
#wgnNoise = 0.05 * np.random.standard_normal(t.shape)
#x += wgnNoise
#c, f = cwt(x, 'morl', sampling_frequency=fs)


In [ ]:
scale = range(1,33)
fs = 1000
train_cwt_coeffs = np.zeros((len(X_train_resamp),len(scale),256))
#train_cwt_coeffs, freqs = pywt.cwt(X_train, scale, 'morl')
for i in range(len(X_train_resamp)):
    #[train_cwt_coeffs[i], freqs] = pywt.cwt(X_train_resamp[i], scale, 'morl')
    [train_cwt_coeffs[i], freqs] = cwt(X_train_resamp[i], 'morl',fs)


test_cwt_coeffs = np.zeros((len(X_test_resamp),len(scale),256))
#train_cwt_coeffs, freqs = pywt.cwt(X_train, scale, 'morl')
for i in range(len(X_test_resamp)):
    #[test_cwt_coeffs[i], freqs] = pywt.cwt(X_test_resamp[i], scale, 'morl')
    [test_cwt_coeffs[i], freqs] = cwt(X_test_resamp[i], 'morl',fs)

del X_train_resamp
del X_test_resamp

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: ComplexWarning: Casting complex values to real discards the imaginary part
  import sys
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: ComplexWarning: Casting complex values to real discards the imaginary part
  


In [ ]:
def get_model():
    nclass = 4
    inp = tf.keras.layers.Input(shape=(32, 256, 1))
    img_1 = Conv2D(16,3, activation=tf.keras.activations.relu, padding="same")(inp)
    img_1 = MaxPool2D(pool_size=(2,2))(img_1)
    img_1 = Conv2D(32,3, activation=tf.keras.activations.relu, padding="same")(img_1)
    img_1 = MaxPool2D(pool_size=(2,2))(img_1)
    img_1 = Dropout(rate=0.2)(img_1)
    img_1 = Flatten()(img_1)
    dense_1 = Dense(16, activation=tf.keras.activations.relu, name="dense_1")(img_1)
    dense_1 = Dense(8, activation=tf.keras.activations.relu, name="dense_2")(dense_1)
    dense_1 = Dense(nclass, activation=tf.keras.activations.softmax, name="dense_3_mitbih")(dense_1)

    model = tf.keras.models.Model(inputs=inp, outputs=dense_1)
    opt = tf.keras.optimizers.Adam(0.001)
    #opt = optimizers.Adam(0.001)

    model.compile(optimizer=opt, loss=tf.keras.losses.sparse_categorical_crossentropy, metrics=['acc'])
    #model.compile(optimizer=opt, loss=tf.keras.losses.categorical_crossentropy, metrics=['acc'])
    model.summary()
    return model

In [ ]:
model = get_model()
file_path = "ECG_mitbih.h5"
checkpoint = tf.keras.callbacks.ModelCheckpoint(file_path, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
early = tf.keras.callbacks.EarlyStopping(monitor="val_acc", mode="max", patience=5, verbose=1)
redonplat = tf.keras.callbacks.ReduceLROnPlateau(monitor="val_acc", mode="max", patience=3, verbose=2)
callbacks_list = [checkpoint, early, redonplat]  # early

model.fit(train_cwt_coeffs, Y_train_init, epochs=25, verbose=1, callbacks=callbacks_list, validation_split=0.1)
#model.load_weights(file_path)

pred_test = model.predict(test_cwt_coeffs)
pred_test = np.argmax(pred_test, axis=-1)

f1 = f1_score(Y_test_init, pred_test, average="macro")

print("Test f1 score : %s "% f1)

acc = accuracy_score(Y_test_init, pred_test)

print("Test accuracy score : %s "% acc)

cm = confusion_matrix(Y_test_init, pred_test)
cm = cm / np.sum(cm,axis=0)

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 32, 256, 1)]      0         
                                                                 
 conv2d (Conv2D)             (None, 32, 256, 16)       160       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 16, 128, 16)      0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 16, 128, 32)       4640      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 8, 64, 32)        0         
 2D)                                                             
                                                                 
 dropout (Dropout)           (None, 8, 64, 32)         0     

In [ ]:
MODEL_DIR = 'intmodel/'
MODEL_TFLITE = MODEL_DIR + 'avgint8model.tflite'

In [ ]:
model.save('intmodel/model.h5')
model.save('intmodel/model')

In [ ]:
train_cwt_coeffs[0].shape

(32, 256)

In [ ]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 32, 256, 1)]      0         
                                                                 
 conv2d (Conv2D)             (None, 32, 256, 16)       160       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 16, 128, 16)      0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 16, 128, 32)       4640      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 8, 64, 32)        0         
 2D)                                                             
                                                                 
 dropout (Dropout)           (None, 8, 64, 32)         0     

In [ ]:
train_cwt_coeffs[0].dtype

dtype('float64')

In [ ]:
import tensorflow as tf
# Convert the model to the TensorFlow Lite format with quantization
converter = tf.lite.TFLiteConverter.from_saved_model('intmodel/model')
# Set the optimization flag.
converter.optimizations = [tf.lite.Optimize.DEFAULT]

# Enforce integer only quantization
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
converter.inference_input_type = tf.int8
converter.inference_output_type = tf.int8
# Provide a representative dataset to ensure we quantize correctly.
def representative_dataset():
    for i in range(500):
        yield([np.float32(train_cwt_coeffs[i]).reshape(1, 32,256,1)])


converter.representative_dataset = representative_dataset
model_tflite = converter.convert()

# Save the model to disk
open(MODEL_TFLITE, "wb").write(model_tflite)

272432